In [12]:
import csv
import sys
import os
import pickle
import numpy as np
import pandas as pd
import tqdm as tqdm
import imageio
from PIL import Image

In [13]:
# Level sorting for household funiture and electrical devices , tree
CIFAR100_FIND_Household_LABLES = ["clock","computer","keyboard","lamp","telephone","television","bed","chair","couch","table","wardrobe"]
CIFAR100_FIND_Tree1_LABLES = ["maple_tree","oak_tree","palm_tree"]
CIFAR100_Householld_dic = {"clock":"clock","computer":"computer","keyboard":"keyboard","lamp":"lamp","telephone":"telephone","television":"television","bed":"bed","chair":"chair","couch":"couch","table":"table","wardrobe":"wardrobe",   }
CIFAR100_Tree1_dic ={"maple_tree":"maple_tree","oak_tree":"oak_tree","palm_tree":"palm_tree"}

In [14]:
def get_labels(dic_files):
    labels = [key for key in dic_files.keys()]
    labels.sort()
    return labels

def unpickle(file):
    """unpickle file into a dictornary"""
    with open(file,'rb') as fo:
        res = pickle.load(fo,encoding='bytes')
    return res

def cifar2rgb(line):
    return Image.fromarray(np.transpose(np.reshape(line,[32,32,3],order='F'),[1,0,2]))

In [15]:
def cifar100(dest,dictonary,labels):
    current_dir=os.path.abspath(os.getcwd())
    for label in labels:
        os.mkdir(dest + '/' + dictonary[label])
        
    counters = { label:1 for label in labels}
    
    #meta file unpickle
    meta_dic = unpickle(current_dir + '/meta')
    fine_labels = meta_dic[b'fine_label_names']
    coarse_labels = meta_dic[b'coarse_label_names']
    
    #train file unpickle
    batch = unpickle(os.path.abspath(os.getcwd()) + '/train')
    
    for idx, line in enumerate(batch[b'data']):
        fine_label = fine_labels[batch[b'fine_labels'][idx]].decode('utf8')
        coarse_label = coarse_labels[batch[b'coarse_labels'][idx]]
        
        is_fine = fine_label in labels
        if is_fine:
            image = cifar2rgb(line)
            label = fine_label if is_fine else coarse_label
            image.save(dest + "/" + dictonary[label] + "/" + str(counters[label]) + ".png")
            counters[label] += 1

In [16]:
def write_csv(dest,filename,dictonary,labels):
    """Writes the filename.csv file, reading dest content that"""
    with open( filename + ".csv", "w+") as csv_file:
        writer = csv.DictWriter(csv_file, ["file", "label"])
        writer.writeheader()

        for label in get_labels(dictonary):
            hosuehold_label = dictonary[label]
            label_id = labels.index(hosuehold_label)

            path = dest + "/" + hosuehold_label + "/"
            images_count = len(next(os.walk(path))[2]) + 1
            for image in range(1, images_count):
                relative_path = "DataSet/" + hosuehold_label + "/" + str(image) + ".png"
                writer.writerow({"file": relative_path, "label": hosuehold_label})

In [17]:
def Household(dest):
    filename = 'Test_Household'
    cifar100(dest,CIFAR100_Householld_dic,CIFAR100_FIND_Household_LABLES)
    write_csv(dest,filename,CIFAR100_Householld_dic,CIFAR100_FIND_Household_LABLES)
    
def Tree_1(dest):
    filename = 'Test_Tree'
    cifar100(dest,CIFAR100_Tree1_dic,CIFAR100_FIND_Tree1_LABLES)
    write_csv(dest,filename,CIFAR100_Tree1_dic,CIFAR100_FIND_Tree1_LABLES)


In [18]:
def main():
    current_dir = os.path.abspath(os.getcwd())
    dest = current_dir + "/DataSet"
    if not os.path.exists(dest):
        os.mkdir(dest)
        Household(dest)
        Tree_1(dest)   
    else:
        print("DateSet already built. Exit")

In [19]:
if __name__ == '__main__':
    main()